In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix

train = False

path = "dataset/normalized/"

df = pd.read_csv(path + "kart.csv")

# Split the data into features (X) and target (y)
X = df.drop('action', axis=1)
y = df['action']

# Split the data into training and test sets
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.transform(y_val)

# Número de clases
num_classes = 8

# Convertir a codificación one-hot para usar MSE
y_train = tf.keras.utils.to_categorical(y_train_encoded, num_classes=num_classes, dtype='float32')
y_val = tf.keras.utils.to_categorical(y_val_encoded, num_classes=num_classes, dtype='float32')

if (train):
    early_stopping = EarlyStopping(
        monitor='val_loss',  # Métrica a monitorear (puede ser 'val_accuracy', 'val_loss', etc.)
        patience=10,          # Número de épocas sin mejora antes de detener el entrenamiento
        restore_best_weights=True  # Restaura los pesos del modelo al mejor logrado durante el entrenamiento
    )

    model = keras.Sequential()

    model.add(layers.Dense(64, activation='relu', input_shape=(8,)))

    # Agrega una capa de Reshape para agregar la dimensión de los pasos de tiempo
    model.add(layers.Reshape((1, 64)))

    # Capa LSTM
    model.add(layers.LSTM(64))

    # Capa de salida
    model.add(layers.Dense(num_classes, activation='linear'))

    model.summary()

    model.compile(
        loss=keras.losses.MeanSquaredError(),
        optimizer="sgd",
        metrics=["accuracy"],
    )

    history = model.fit(
        x_train,
        y_train,
        batch_size=16,
        epochs=5000,
        validation_data=(x_val, y_val),
        callbacks=[early_stopping]
    )

    # Guardar el modelo en formato keras
    model.save('rnn_entrenado.keras')
    
else:
    from tensorflow.keras.models import load_model
    
    model = load_model('rnn_entrenado.keras')
    

y_pred = model.predict(x_val)

predicted_classes = y_pred.argmax(axis=1)

accuracy = accuracy_score(y_val_encoded, predicted_classes)
precision = precision_score(y_val_encoded, predicted_classes, average='weighted', zero_division=1)
recall = recall_score(y_val_encoded, predicted_classes, average='weighted')
conf_matrix = confusion_matrix(y_val_encoded, predicted_classes)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("\nConfusion Matrix:")
print(conf_matrix)




47/47 [==============================] - 1s 2ms/step
Accuracy: 0.7086720867208672
Precision: 0.7220692057772851
Recall: 0.7086720867208672

Confusion Matrix:
[[608 196   0]
 [167 438   0]
 [ 61   6   0]]
